# Mount Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop, SGD, Adagrad, Adadelta, Adam, Adamax, Nadam
from sklearn.metrics import confusion_matrix
import itertools

# Dataset

In [ ]:
!cp /content/drive/MyDrive/HCMUS/CDTGMT/Project2/FashionMNIST/fashion-mnist_test.csv /content

In [ ]:
!cp /content/drive/MyDrive/HCMUS/CDTGMT/Project2/FashionMNIST/fashion-mnist_train.csv /content

In [ ]:
train = pd.read_csv("fashion-mnist_train.csv")
test = pd.read_csv("fashion-mnist_test.csv")

# Preprocess Data 

In [ ]:
labels = ["T-shirt/top","Trouser", "Pullover", "Dress","Coat",
       "Sandal", "Shirt","Sneaker","Bag", "Ankle Boot"]

height = 28
width = 28
channel = 1

In [ ]:
def preprocess_data(data):
    y_output = to_categorical(data.label, num_classes=10, dtype="float32")
    x_output = np.array(data.values[:,1:])
   
    #Normalize data
    x_output = x_output.reshape(data.shape[0], height, width, channel)
    x_output = np.pad(x_output, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    x_output = x_output/255
    
    return x_output, y_output

In [ ]:
X_train, Y_train = preprocess_data(train)
X_test, Y_test = preprocess_data(test)

# Model 

In [ ]:
# LeNet5
model = Sequential()
model.add(Conv2D(filters = 6, 
                  kernel_size = 5, 
                  strides = 1, 
                  activation = 'relu', 
                  input_shape = (32,32,1))) 
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Conv2D(filters =16 , 
                  kernel_size = 5,
                  strides = 1,
                  activation = 'relu',
                  input_shape = (14,14,6)))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Flatten())
model.add(Dense(units = 120, activation = 'relu'))
model.add(Dense(units = 84, activation = 'relu'))
model.add(Dropout(0.5)) 
model.add(Dense(units = 10, activation = 'softmax'))
model.compile(optimizer =Adamax(learning_rate=0.005), # optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])
model.summary()

#callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='accuracy', 
    patience=5, 
    verbose=1, 
    factor=0.5, 
    min_lr=1e-6) 

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="loss",
    min_delta=0, 
    patience=10, 
    verbose=0,
    mode="auto") 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 6)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 120)               4

# Train Model  

In [ ]:
#Train Model
train_model = model.fit(
    X_train, Y_train,
    batch_size=32,
    steps_per_epoch = 1500,
    epochs=10,
    verbose=1,
    callbacks=[reduce_lr, early_stop])

Epoch 1/10
1500/1500 [==============================] - 35s 22ms/step - loss: 0.5870 - accuracy: 0.7911 - lr: 0.0050
Epoch 2/10
1500/1500 [==============================] - 32s 22ms/step - loss: 0.3949 - accuracy: 0.8620 - lr: 0.0050
Epoch 3/10
1500/1500 [==============================] - 33s 22ms/step - loss: 0.3454 - accuracy: 0.8776 - lr: 0.0050
Epoch 4/10
1500/1500 [==============================] - 32s 21ms/step - loss: 0.3205 - accuracy: 0.8866 - lr: 0.0050
Epoch 5/10
1500/1500 [==============================] - 32s 21ms/step - loss: 0.2956 - accuracy: 0.8940 - lr: 0.0050
Epoch 6/10
1500/1500 [==============================] - 32s 21ms/step - loss: 0.2805 - accuracy: 0.9004 - lr: 0.0050
Epoch 7/10
1500/1500 [==============================] - 32s 21ms/step - loss: 0.2624 - accuracy: 0.9059 - lr: 0.0050
Epoch 8/10
1500/1500 [==============================] - 32s 21ms/step - loss: 0.2556 - accuracy: 0.9081 - lr: 0.0050
Epoch 9/10
1500/1500 [==============================] - 32s 21ms

In [ ]:
#predict
y_pred = model.predict(X_test)
test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss    :', test_loss)
print('Test accuracy:', test_acc)

Test loss    : 0.25102099776268005
Test accuracy: 0.90829998254776
